In [ ]:
!pip install torch syft

In [ ]:
!python -m venv syft_env
!source syft_env/bin/activate  # Activate the environment

In [ ]:
!pip install --upgrade pip==24.0.2

In [ ]:
import torch
import syft as sy
from torch import nn, optim

# Initialize the virtual workers (simulating multiple devices)
hook = sy.TorchHook(torch)
worker1 = sy.VirtualWorker(id="worker1", hook=hook)
worker2 = sy.VirtualWorker(id="worker2", hook=hook)

# Dummy model for recommendation
class RecommenderModel(nn.Module):
    def __init__(self):
        super(RecommenderModel, self).__init__()
        self.fc = nn.Linear(10, 1)  # Example with 10 features for simplicity

    def forward(self, x):
        return self.fc(x)

# Model initialization
model = RecommenderModel()

# Federated data simulated
data1 = torch.randn(10).send(worker1)
data2 = torch.randn(10).send(worker2)

target1 = torch.tensor(1.0).send(worker1)
target2 = torch.tensor(0.0).send(worker2)

# Federated Training Loop
optimizer = optim.SGD(model.parameters(), lr=0.1)
loss_fn = nn.MSELoss()

# Training over the federated data
for epoch in range(10):
    # Send the model to each worker
    model.send(worker1)

    # Training on worker1
    optimizer.zero_grad()
    prediction1 = model(data1)
    loss1 = loss_fn(prediction1, target1)
    loss1.backward()
    optimizer.step()

    # Get the model back to the main machine before sending to worker2
    model.get()

    # Send the model to worker2
    model.send(worker2)

    # Training on worker2
    optimizer.zero_grad()
    prediction2 = model(data2)
    loss2 = loss_fn(prediction2, target2)
    loss2.backward()
    optimizer.step()

    # Get the model back to the main machine
    model.get()

    print(f"Epoch {epoch}, Loss1: {loss1.item()}, Loss2: {loss2.item()}")


In [ ]:
!pip install pycryptodome

In [ ]:
from Crypto.Cipher import AES
import base64

def encrypt_data(data, key):
    cipher = AES.new(key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(data.encode())
    return base64.b64encode(cipher.nonce + tag + ciphertext).decode()

def decrypt_data(encrypted_data, key):
    encrypted_data = base64.b64decode(encrypted_data)
    nonce, tag, ciphertext = encrypted_data[:16], encrypted_data[16:32], encrypted_data[32:]
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    decrypted_data = cipher.decrypt_and_verify(ciphertext, tag)
    return decrypted_data.decode()

# Example usage
key = b'Sixteen byte key'  # 16-byte key for AES
user_data = "User sensitive health data"
encrypted = encrypt_data(user_data, key)
decrypted = decrypt_data(encrypted, key)

print(f"Encrypted: {encrypted}")
print(f"Decrypted: {decrypted}")

In [ ]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Sample data (users' workout preferences)
data = {'user_id': [1, 2, 3, 4, 5],
        'workout_preference': [5, 3, 4, 2, 1],
        'diet_preference': [3, 4, 2, 5, 3]}

df = pd.DataFrame(data)

# Create a simple recommender system using Nearest Neighbors
X = df[['workout_preference', 'diet_preference']].values
model = NearestNeighbors(n_neighbors=2, algorithm='ball_tree')
model.fit(X)

# Recommend similar users for user 1
distances, indices = model.kneighbors([X[0]])
print(f"Recommendations for user 1: {indices}")